# Tree Methods for Moving Interface

这篇文章主要由 John Strain 描述了对于移动界面一个新的数值方法，提出了对于解移动界面问题的快速适应方法.这个方法结合了一个水平集方法，半拉格朗日时间层上的格式和四叉树网格.这个树方法重解和移动复杂的界面在优化时间上都是没有被限制的，它是无条件稳定的;对于移动界面的一个黑盒子方法，对于界面问题的解是简单的； 一个四叉树网格被用于在界面上的计算，因此这个方法在每一个时间层上带有 $N$ 个自由度；对于抛物问题也可以用大的时间步长.　

未来的发展计划：
1. 对抛物问题 CFL 条件的研究
1. 时间层上的高阶精度
1. 完成移动界面方法的模块化
1. 应用工业结晶增长问题

这个方法通过由一个水平集方法合并并且打破了界面的自动化；四叉树网格重解高效的优化界面：在每一个时间层上移动一个 $N$ 单元的界面；带有无条件稳定的半拉格朗日格式允许大的时间步长；快速重新定距算法保持了一个数值近似的稳健性.

## 移动界面和水平集

### 移动界面

一个一般的移动界面是一个依赖于时间 $t$ 集合 $\Omega(t)\subset R^d$ 的边界 $\Gamma(t)=\partial \Omega(t)$. 如果 $\Omega$ 足够光滑，则 $\Gamma(t)$ 在每一个节点上有 外单位法向量 $N$ 和一个法向速度 $V$; 移动界面问题是一个接近于把 $V$ 作为 $\Gamma(t)$ 的一个函数的方程系统.

*Passive transport*

在 $R^d$ 上给出的一个速度场 $F(x,t)$ 和带有法向速度 $V=N\cdot F$ 的界面 $\Gamma(t)$

*Unit normal velocity*

最简单的几何流的移动界面带有的法向速度是 $V=1$

*Anisotropic curvature-dependent velocity*

一个更一般的几何运动有法向速度

$$
V(x,t) = R + \epsilon \cos(K\theta+\theta_0)+(R'+\epsilon'\cos(K'\theta+\theta_0'))C
$$

这里 $\cos \theta = N\cdot e_1$ 是法向量与 $x$ 轴正向的夹角的余弦值.

*Crystal growth*

许多工业问题包含了两个不同相之间的移动界面,这个界面在固相和液相或者气相之间增长.例如，Stefan 问题
$$
u_t = \nabla u\qquad off~\Gamma(t)\\
u = -\epsilon C\qquad on ~\Gamma(t)
$$

这里速度场 $u$ 是未知的，带有法向速度 $V$ 的界面 $\Gamma$　等于 $u$ 的法向导数

### 水平集方法

移动界面主要的困难是正确对界面合并，破坏，和其他拓扑改变的处理. 可以通过把移动界面用水平集方程表示来克服这个困难.　用任一函数 $\varphi:R^d\times R\to R$ 的零水平集

$$
\Gamma(t) = \{x\in R^d:\varphi(x,t)=0\}
$$

符号距离函数
$$
\varphi(x,t) = \pm \min_{y\in \Gamma(t)}||x-y||
$$


外单位法向量
$$
N=\nabla \varphi/||\nabla \varphi||
$$

法向速度
$$
V=\varphi_t/||\nabla\varphi||
$$

给出一个法向速度 $VN$ 到函数 $F(x,t)$ 扩展，水平集方程界面通过 $\varphi$ 的变化移动

$$
\varphi_t - F\cdot\nabla \varphi = \phi_1-(F\cdot N)||\nabla\varphi||=0
$$

这个方法对于界面嵌入到 $\varphi$ 上而不是 $\Gamma(t)$ 上的拓扑,并且自动的处理合并，破坏，和其他的拓扑改变的情况.

*Passive transport*

水平集方程已经成为一个线性双曲偏微分方程

$$
\phi_t-F(x,t)\cdot\nabla\varphi=0
$$

*Unit normal velocity*

带有单位法向速度的运动成为非线性双曲偏微分方程

$$
\varphi_t-||\nabla\varphi||=0
$$

*curvature-dependent velocity*

法向速度

$$
\varphi_t -(R + \epsilon \cos(K\theta+\theta_0))||\nabla\varphi||=(R'+\epsilon'\cos(K'\theta+\theta_0'))\nabla\cdot(\nabla \varphi/||\nabla\varphi||)||\nabla\varphi||C
$$

这里 $\cos \theta =\varphi_x/||\nabla\varphi||$ ,$C=-\nabla\cdot N$

**水平集方法的几个缺点**
1. 对于高维情况需要更大的消耗
1. 速度的扩展更困难
1. 方法不再是模块化：对于每一个新的问题必须得重新写一个新的代码

**优点**

可以获得正确的粘性解，方法的适应性是消除增加的消耗.

### 半拉格朗日水平集方法

这个方法是文件的，精确地，但是对于远离界面的非一致网格需要更大的消耗.文章在界面附近完成了半拉格朗日水平集方法在四叉树网格的计算，得到精确度的前提下比非一致网格的消耗更少.

**The CIR Scheme**

线性双曲偏微分方程

$$
\varphi_t-F(x,t)\cdot\nabla\varphi=0
$$

通过曲率方程

$$
\dot{s(t)}=-F(s(t),t)
$$

传递 $\varphi$ 的值

给出在时间$t_n$ 的 $\varphi$ ,CIR 在时间层 $t_{n+1} = t_n+k$ 上任一点 $x$ 通过评估速度 $F(x,t)$ 近似 $\varphi(x,t_{n+1})$ ,典型的向后近似通过直线

$$
x+(t_{n+1}-t)F(x,t_n)\thickapprox s(t)
$$

在时间 $t_n$ 上 $\varphi$ 的线性插值

$$
x+kF(x,t_n)\thickapprox s(t_n)
$$

**收敛**

CIR 的截断误差 

$$
\tau = O\left(\frac{h^2}{k}\right)+O(k)
$$

**半拉格朗日水平集方法**

优点：无条件稳定的，并且满足 CFL 条件，即使对于大步长 $k = O(h)$ 的抛物问题也允许收敛.重新定距和速度扩展技术也完成了大步长的信息转换且有助于满足CFL 条件

## 树方法

移动界面通过结合下边的思想发展树方法：
1. 拓扑改变要求水平集方程的解仅仅是在局部界面附近，而不是全局空间
1. 通过一个四叉树网格可以以最佳消耗解界面问题
1. 半拉格朗日时间步长格式允许步长由解的要求决定，而不是数值稳定性决定
1. 半拉格朗日格式对网格节点都是独立的计算，允许没有迭代的自适应加密
1. 频繁地重新定距，对于建立一个四叉树网格给出一个分割标准，并且作出误差估计

总结这个方法：
1. 初始化水平集方程的解 $\varphi$：给出一个初始界面 $\Gamma = \Gamma(0)$ ,建立一个四叉树 $Q_0$ ,且在 $Q_0$ 上近似符号距离函数 $\varphi_0$
1. 在一个时间步长上发展界面:给一个四叉树 $Q_n$ 重解 $\varphi_n\thickapprox\varphi(t_n)$ 的零集合 $\Gamma_n$ , 扩展速度 $F_n$ 等于法向速度 $VN$ ,建立一个四叉树 $Q_{n+1}$ 解CIR 近似的零集合 $\Gamma_{n+1}$

$$
\varphi_{n+1}(x) = \varphi_n(x+kF_n(x))
$$

计算 $\varphi_{n+1}$ 包含四个过程：
1. 扩展
1. 重解
1. 插值
1. 边界条件的应用

重新定距可以消除数值影响，提升数值精度.重新定距等同于初始化，一旦 $\Gamma$ 被找到，则找到 $\Gamma$ 的等高线可用. 最简单的技术是将单元分成两个三角形，在每一个三角形上找到线性内插 $\varphi$ 的精确零段，然后添加到界面，单元直接分割的选择使得这些等高线各向异性并且有助于显示误差:通过显示方向的偏差计算未解决的符号误差

水平集方法要求一个全局定义的速度 $F$. 树方法也要求速度的扩展.  对于正向传输 $F$ 已经给出，而几何速度

$$
F=(R+\epsilon(K\theta+\theta_0))N+ (R'+\epsilon'(K'\theta+\theta_0'))CN
$$

可以由几何形式
$$
N=\nabla\varphi/||\nabla\varphi|| \\
C = -\nabla\cdot N
$$

估计

扩展几何速度产生两个困难
1. 当面或拐角发展时，精确解不可微分，∇φ在极值处消失，因此N和C在这里没有定义，并且在四叉树上的重新定距以及引入不连续性
1. CFL 条件，即对于水平集方程在几乎所有的显示格式要求小的时间步长

对于几何速度可以基于单元格式和基于网格格式来估计. 

基于单元格式是快速的，对于 $\varphi$ 的一阶导数问题比较好；而基于网格格式是比较慢的，更一般的，对于曲率依赖速度问题比较好.

## 重解

这个方法建立一个四叉树网格来解CIR近似

$$
\varphi_{n+1}(x) = \varphi_n(x+kF_n(x))
$$

到水平集函数　$\varphi(x,t_{n+1})$


通过下边的标准分割从根单元递归建立一个四叉树:

分割每个 $|\varphi_{n+1}|$ 大于边长的单元

在每步的重新定距

$$
\varphi_{n+1}(x) = \varphi_n+kF_n\cdot\nabla\varphi_{n}+o(k)=\varphi_n+O(k)
$$

## 插值

CIR 格式要求在点 $s = x+kF_n(x)$ 上的插值，许多一般的插值技术都是可用的，但是由于四叉树 $Q_n$ 的不规则性，选择就会受到限制，并且有两个要求:
1. 水平集函数 $\varphi$ 是李普希茨连续的，这样高阶精度的方法就会避免要求光滑性
1. 在任意给出的形式，半拉格朗日的稳定性方法都是被保证的

在单元 $C$ 上双线性插值到点

$$
(1-\alpha)(1-\beta)\varphi_{00}+\alpha(1-\beta)\varphi_{10}+(1-\alpha)\beta\varphi_{01}+\alpha\beta\varphi_{11}
$$

$C$ 的顶点值由

$$
\varphi_{ij}=\varphi(x_0+ih,y_0+jh)
$$



双线性插值保持了 CIR 格式的最大值原则，并且保持了局部二阶精度，在 $O(1/k)$ 时间层之后的全局一阶误差

## 边界条件

当 $s$ 位于区域外部，CIR 格式要求数值边界条件指定 $\varphi(s,t_n)$

有两个简单的边界条件：扩展和投影. 在扩展中，把 $\varphi$ 扩展成一个常数或者线性函数，然后应用标准插值格式插值 $s$ 的扩展值; 在投影中，当 $s$ 离开区域时阻止 $s$ 并且当 $s$ 通过边界时，用一边插值到点

树方法用投影是因为它是简单的并且是有效的